In [743]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pydotplus
from sklearn.ensemble import RandomForestClassifier
# from sklearn.externals import 
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
import catboost

In [719]:
model=DecisionTreeClassifier(min_samples_leaf=10, max_depth=6, min_impurity_decrease=0.001)

In [720]:
features=pd.read_csv('BigCapFeatures.csv')
# targets=pd.concat([pd.read_csv('momentumeEffect.csv'), pd.read_csv("momentumeEffect5.csv")])
targets=pd.read_csv('BigCapMomentum.csv')
targets.drop_duplicates(['ticker'],keep='first',inplace=True)

In [721]:
preYearFeatures=pd.read_csv('BigCapMomentum2017.csv')

In [722]:
preYearFeatures.head(20)

,Unnamed: 0,ticker,min_1,max_1,var_1,max_1_peroids,buyHold_1,maxDrawBack
0,0,A,-0.087603,0.138202,2.927907,"(13, 60)",46.962768,6.217640
1,0,ABMD,0.000000,0.237624,3.209953,"(1, 60)",69.296377,6.028203
2,0,AAP,-0.070361,0.116333,2.797046,"(11, 50)",-41.532967,5.674570
3,0,ABBV,0.000000,0.297133,8.474027,"(1, 50)",55.975846,5.946876
4,0,ABT,0.000000,0.230023,7.278602,"(1, 90)",49.106904,4.524829
5,0,AAPL,-0.171143,0.119133,5.805323,"(3, 100)",47.253886,7.795955
6,0,ABB,-0.045695,0.153179,3.378855,"(13, 50)",26.631983,4.196955
7,0,ABEV,-0.114269,0.104585,4.432026,"(5, 50)",27.075097,10.711745
8,0,ACAD,-0.245348,0.445953,15.732869,"(17, 50)",7.037423,7.487288
9,0,ABC,-0.331285,0.178803,6.730906,"(13, 50)",18.316134,7.121015


In [723]:
preYearFeatures['preYearMomentum']=preYearFeatures[['min_1','max_1','var_1']].apply(lambda x: 1 if (x[0]+x[1]>=0 and x[1]>=0.15 and x[2]<=12 ) else (-1 if (x[0]<= -0.15 and x[2]<=12) else 0), axis=1)
preYearFeatures.rename({'ticker':'Symbol'}, axis=1, inplace=True)

In [732]:
preYearFeatures.head(3)

,Unnamed: 0,Symbol,min_1,max_1,var_1,max_1_peroids,buyHold_1,maxDrawBack,preYearMomentum
0,0,A,-0.087603,0.138202,2.927907,"(13, 60)",46.962768,6.217640,0
1,0,ABMD,0.000000,0.237624,3.209953,"(1, 60)",69.296377,6.028203,1
2,0,AAP,-0.070361,0.116333,2.797046,"(11, 50)",-41.532967,5.674570,0


In [725]:
features

,Unnamed: 0,Symbol,PE,EPSGrowth,RevenueGrowth,logMarketCap,PS,Sector,Sharpe,std,LiquidityRate,CorWithBenchMark
0,0,A,28.447257,0.196970,0.064255,10,4.684263,Healthcare,2.675440,0.164655,0.520762,0.437042
1,0,AA,17.960133,2.409606,0.250483,10,0.945171,Basic Materials,2.329636,0.372822,1.436134,0.346086
2,0,AACG,-7.536467,-0.000000,0.037413,8,0.329082,Consumer Defensive,0.343638,0.502240,0.036839,0.145524
3,0,AAIC,5.175439,-0.170909,10.125368,8,5.071731,Real Estate,-1.100027,0.199405,1.396349,0.241527
4,0,AAL,10.679592,-0.140351,0.050448,10,0.793000,Industrials,0.431699,0.286677,0.825448,0.439798
...,...,...,...,...,...,...,...,...,...,...,...,...
1196,0,TSLA,-36.194896,4.354037,0.679790,10,5.645217,Consumer Cyclical,1.228039,0.354108,3.030228,0.170339
1197,0,TSM,18.516317,0.088458,0.031156,11,0.218991,Technology,2.106366,0.167483,0.094604,0.366633
1198,0,TSN,14.669078,0.144928,0.037391,10,0.789125,Consumer Defensive,1.513022,0.195279,0.570319,0.213075
1199,0,TT,15.400907,0.094431,0.050981,10,1.188853,Industrials,1.037914,0.168255,0.872713,0.493571


In [726]:
targets

,Unnamed: 0,ticker,min_1,max_1,var_1,max_1_peroids,buyHold_1,maxDrawBack
0,0,A,-0.336212,-0.045040,6.318015,"(11, 140)",-1.601901,13.808617
1,0,AAL,-0.286046,0.098800,9.119530,"(5, 50)",-38.677625,11.761279
2,0,AAP,-0.042957,0.420985,11.629514,"(3, 80)",54.658078,8.799882
3,0,AAPL,0.022848,0.297409,5.858871,"(3, 60)",-6.834746,8.690860
4,0,ABB,-0.091169,0.096623,3.066543,"(1, 180)",-29.536181,2.552621
...,...,...,...,...,...,...,...,...
936,0,TSLA,-0.518498,0.009894,13.671449,"(5, 190)",8.266021,16.473924
937,0,TSM,-0.147707,0.123101,5.220360,"(1, 60)",-7.603056,6.495531
938,0,TSN,-0.096906,0.065409,2.272394,"(1, 90)",-35.207101,5.777786
939,0,TT,-0.148300,0.083426,5.098414,"(1, 100)",1.783138,7.651097


In [727]:
targets['MomentumEffect']=targets[['min_1','max_1','var_1']].apply(lambda x: 1 if (x[0]+x[1]>=0 and x[1]>=0.15 and x[2]<=12 ) else (-1 if (x[0]<= -0.15 and x[2]<=12) else 0), axis=1)

In [728]:
targets['var_1'].quantile([0.25, 0.5, 0.75])

0.25    4.466506
0.50    5.856409
0.75    7.798617
Name: var_1, dtype: float64

In [733]:
targets.rename({'ticker':'Symbol'}, axis=1, inplace=True)

In [734]:
combined=pd.merge(features, targets, how='inner', on=['Symbol'])
combined=pd.merge(combined, preYearFeatures[['Symbol','preYearMomentum']], how='inner', on=['Symbol'])

In [741]:
combined['Year']=2018
combined

,Unnamed: 0_x,Symbol,PE,EPSGrowth,RevenueGrowth,logMarketCap,PS,Sector,Sharpe,std,...,Unnamed: 0_y,min_1,max_1,var_1,max_1_peroids,buyHold_1,maxDrawBack,MomentumEffect,preYearMomentum,Year
0,0,A,28.447257,0.196970,0.064255,10,4.684263,Healthcare,2.675440,0.164655,...,0,-0.336212,-0.045040,6.318015,"(11, 140)",-1.601901,13.808617,-1,0,2018
1,0,AAL,10.679592,-0.140351,0.050448,10,0.793000,Industrials,0.431699,0.286677,...,0,-0.286046,0.098800,9.119530,"(5, 50)",-38.677625,11.761279,-1,-1,2018
2,0,AAP,18.754647,-0.246499,-0.020266,9,0.719693,Consumer Cyclical,-1.026959,0.404739,...,0,-0.042957,0.420985,11.629514,"(3, 80)",54.658078,8.799882,1,0,2018
3,0,AAPL,17.488181,0.165269,0.063045,11,3.211131,Technology,2.591127,0.176369,...,0,0.022848,0.297409,5.858871,"(3, 60)",-6.834746,8.690860,1,-1,2018
4,0,ABB,26.165049,0.119565,0.014308,10,1.576513,Industrials,1.793062,0.145192,...,0,-0.091169,0.096623,3.066543,"(1, 180)",-29.536181,2.552621,0,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,0,TSLA,-36.194896,4.354037,0.679790,10,5.645217,Consumer Cyclical,1.228039,0.354108,...,0,-0.518498,0.009894,13.671449,"(5, 190)",8.266021,16.473924,0,-1,2018
907,0,TSM,18.516317,0.088458,0.031156,11,0.218991,Technology,2.106366,0.167483,...,0,-0.147707,0.123101,5.220360,"(1, 60)",-7.603056,6.495531,0,0,2018
908,0,TSN,14.669078,0.144928,0.037391,10,0.789125,Consumer Defensive,1.513022,0.195279,...,0,-0.096906,0.065409,2.272394,"(1, 90)",-35.207101,5.777786,0,1,2018
909,0,TT,15.400907,0.094431,0.050981,10,1.188853,Industrials,1.037914,0.168255,...,0,-0.148300,0.083426,5.098414,"(1, 100)",1.783138,7.651097,0,0,2018


In [742]:
combined.to_csv('Training2018.csv')

In [737]:
combined.loc[:,['preYearMomentum','MomentumEffect']]

,preYearMomentum,MomentumEffect
0,0,-1
1,-1,-1
2,0,1
3,-1,1
4,1,0
...,...,...
906,-1,0
907,0,0
908,1,0
909,0,0


#### Seem like momentum effect does not persist

In [738]:
accuracy_score(combined['preYearMomentum'], combined['MomentumEffect'])

0.3556531284302964

In [635]:
combined.drop_duplicates(['Symbol'], keep='first', inplace=True)

In [636]:
combined['PE_category']=combined['PE'].apply(lambda x: 1 if (x>0 and x<15) else (0 if x>0 and x<40 else -1))

In [637]:
combined['PE_category']=combined['PE'].apply(lambda x: 'High' if (x>0 and x<15) else 'Low')

In [638]:
combined.head(3)

,Unnamed: 0_x,Symbol,PE,EPSGrowth,RevenueGrowth,logMarketCap,PS,Sector,Sharpe,std,...,Unnamed: 0_y,min_1,max_1,var_1,max_1_peroids,buyHold_1,maxDrawBack,MomentumEffect,preYearMomentum,PE_category
0,0,A,28.447257,0.196970,0.064255,10,4.684263,Healthcare,2.675440,0.164655,...,0,-0.336212,-0.045040,6.318015,"(11, 140)",-1.601901,13.808617,No,0,Low
1,0,AAL,10.679592,-0.140351,0.050448,10,0.793000,Industrials,0.431699,0.286677,...,0,-0.286046,0.098800,9.119530,"(5, 50)",-38.677625,11.761279,No,-1,High
2,0,AAP,18.754647,-0.246499,-0.020266,9,0.719693,Consumer Cyclical,-1.026959,0.404739,...,0,-0.042957,0.420985,11.629514,"(3, 80)",54.658078,8.799882,Yes,0,Low


In [639]:
# categorizeColumns=['EPSGrowth', 'PS', 'std', 'CorWithBenchMark', 'logMarketCap','Sharpe']
categorizeColumns=['PS', 'std','CorWithBenchMark','Sharpe', 'RevenueGrowth','LiquidityRate','preYearMomentum']

In [640]:
for col in categorizeColumns:
    quantile=combined[col].quantile(0.5)
    combined[col+'_category']=combined[col].apply(lambda x: 1 if x>= quantile else -1)

In [641]:
combined.replace([np.inf, -np.inf], 0, inplace=True)
combined=combined[combined['PS']<100]
combined.dropna(inplace=True)

In [642]:
# X=combined[categorizeColumns]
X=combined[[cat+'_category' for cat in categorizeColumns] +['PS']]
Y=combined["MomentumEffect"]

In [643]:
# quantile_PS=X['PS'].quantile([0.25, 0.75])
X.head(3)

,PS_category,std_category,CorWithBenchMark_category,Sharpe_category,RevenueGrowth_category,LiquidityRate_category,preYearMomentum_category,PS
0,1,-1,1,1,-1,1,1,4.684263
1,-1,1,1,-1,-1,1,-1,0.793000
2,-1,1,-1,-1,-1,1,1,0.719693


In [644]:
Y.value_counts()

No         348
Yes        287
Neither    262
Name: MomentumEffect, dtype: int64

In [645]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897 entries, 0 to 910
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PS_category                897 non-null    int64  
 1   std_category               897 non-null    int64  
 2   CorWithBenchMark_category  897 non-null    int64  
 3   Sharpe_category            897 non-null    int64  
 4   RevenueGrowth_category     897 non-null    int64  
 5   LiquidityRate_category     897 non-null    int64  
 6   preYearMomentum_category   897 non-null    int64  
 7   PS                         897 non-null    float64
dtypes: float64(1), int64(7)
memory usage: 63.1 KB


In [646]:
pd.DataFrame(Y).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897 entries, 0 to 910
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MomentumEffect  897 non-null    object
dtypes: object(1)
memory usage: 14.0+ KB


In [665]:
X_t, X_val, Y_t, Y_val= train_test_split(X,Y, test_size=0.2)

In [666]:
X_t.head(3)

,PS_category,std_category,CorWithBenchMark_category,Sharpe_category,RevenueGrowth_category,LiquidityRate_category,preYearMomentum_category,PS
731,-1,-1,1,1,1,-1,1,1.961874
215,-1,-1,1,1,-1,-1,1,1.546769
94,1,1,-1,-1,-1,-1,-1,4.898368


In [684]:
model=DecisionTreeClassifier(min_samples_leaf=20, max_depth=20, min_impurity_decrease=0.001)
model.fit(X_t,Y_t)

DecisionTreeClassifier(max_depth=20, min_impurity_decrease=0.001,
                       min_samples_leaf=20)

In [685]:
prediction=model.predict(X_val)
train_predict=model.predict(X_t)
accuracy_score(train_predict, Y_t), accuracy_score(prediction, Y_val)


(0.5285913528591353, 0.4222222222222222)

In [669]:
prediction

array(['Yes', 'Neither', 'No', 'No', 'No', 'Yes', 'Neither', 'No', 'Yes',
       'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Neither', 'No', 'Neither',
       'No', 'No', 'Neither', 'No', 'Yes', 'Neither', 'Yes', 'Yes', 'No',
       'Yes', 'Neither', 'No', 'No', 'Neither', 'Yes', 'No', 'No',
       'Neither', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No',
       'Neither', 'Neither', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No',
       'Neither', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'Neither', 'No',
       'No', 'Yes', 'No', 'Neither', 'Neither', 'Yes', 'No', 'Neither',
       'Neither', 'Yes', 'No', 'Yes', 'No', 'Neither', 'No', 'No',
       'Neither', 'No', 'No', 'Neither', 'Yes', 'Yes', 'Neither', 'No',
       'No', 'No', 'Yes', 'Neither', 'No', 'No', 'Neither', 'Neither',
       'Neither', 'Yes', 'No', 'No', 'Neither', 'Neither', 'No', 'Yes',
       'Yes', 'Yes', 'Neither', 'No', 'No', 'No', 'Neither', 'Neither',
       'No', 'Neither', 'No', 'Yes', 'Yes', 'Neither', 'No', 'No'

In [670]:
dot_data=export_graphviz(model, out_file=None,feature_names= X.columns)
graph=pydotplus.graph_from_dot_data(dot_data)
# graph.write_pdf("dt_tree.pdf")

In [671]:
graph.write_pdf("dt_tree.pdf")

True

In [706]:
forest=RandomForestClassifier(min_samples_leaf=30, max_depth=8, n_jobs=1000, max_features=5)
forest.fit(X_t,Y_t)
pred_forest=forest.predict(X_val)
accuracy_score(pred_forest, Y_val), accuracy_score(forest.predict(X_t), Y_t)

(0.43333333333333335, 0.49511854951185497)

In [673]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 2000), threshold=0.05)
sel.fit(X_t, Y_t)
sel.get_support()

array([False, False, False, False, False, False, False,  True])

In [707]:
X.columns

Index(['PS_category', 'std_category', 'CorWithBenchMark_category',
       'Sharpe_category', 'RevenueGrowth_category', 'LiquidityRate_category',
       'preYearMomentum_category', 'PS'],
      dtype='object')